In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g6pp3zZn8URA610i0BnyyABMlo-1Jxb_DfE_ZMJASeB0nMSpS70HkY
Mounted at /content/drive


In [ ]:
!sudo apt install python-pip
!sudo pip install hazm
!pip install tqdm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python-cryptography-vectors
  python-dbus-dbg python-dbus-doc python-enum34-doc python-gi-cairo
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0 python-fs
  python-gdata python-keyczar python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-

In [ ]:
import pandas as pd
from hazm import *
import sklearn.model_selection
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import re
import itertools
from sklearn.pipeline import Pipeline
import csv
from tqdm import tqdm_notebook as tqdm

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import linear_model


In [ ]:
train_url = 'drive/My Drive/train.csv'
test_url = 'drive/My Drive/test.csv'
result_url = "drive/My Drive/"
stopwords_url = 'drive/My Drive/stopwords.txt'

In [ ]:
def save_result(results, name):
  f = open(result_url+name, "w")
  writer = csv.writer(f)
  writer.writerow(["Id", "Category"])
  for i in range(len(test_df)):
    writer.writerow([i, results[i]])
  f.close()

In [ ]:
def read_data(url):
  df = pd.read_csv(url, engine='python',encoding='utf-8', error_bad_lines=False)
  df.head()
  return df

In [ ]:
def stopwords():
  # stopwords = stopwords_list()
  text_file = open(stopwords_url, "r")
  sw = text_file.read().split('\n')
  return sw

print(stopwords())

['\ufeff!', '"', '#', '%', '\\', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '\\\\', '//', '?', '@', '\\', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '’', '«', '»', '”', '“', '′', '‘', '،', '\\\\', 'و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'گردد', 'حتما', 'سلام', 'دلم', 'دیدن', 'بزنید', 'صحبتهای', 'عنوان', 'معناست', 'خبرهای', 'شب', 'نیست', 'باشد', 'چه', 'عده\u200cای', 'علیه\u200cالسلام', 'باشیم', 'فعلی', 'صد', 'ان', 'انها', 'جناب', 'جناب\u200cاقای', 'میشود', 'عده', 'بنا', 'دکتر', 'عرض', 'بایستی', 'نمایند', 'دست', 'بیش', 'روز\u200cگذشته', 'اشاره', 'میگن', 'کنه', 'مارو', 'اونم', 'اینم', '

# Data : train

 

In [ ]:
df = read_data(train_url)
x_train = df["Text"]
categories = np.array(list(set(df["Category"]))) 
y_train = df["Category"]
y_train_hot = pd.get_dummies(y_train)
print(len(y_train))
print(len(x_train))
print(y_train)
# x_train,x_validation,y_train,y_validation=sklearn.model_selection.train_test_split(x_train, y_train, test_size=0.15)

150096
150096
0              Science and Culture
1                            Sport
2                          Economy
3         Miscellaneous.World News
4                            Sport
                    ...           
150091               Miscellaneous
150092    Miscellaneous.Happenings
150093         Miscellaneous.Urban
150094         Science and Culture
150095                    Politics
Name: Category, Length: 150096, dtype: object


# Preprocessing


In [ ]:
def information(df):
  number_of_each_categories = {}
  for c in categories :
    index = df.index[df["Category"] == c].tolist()
    number_of_rows = len(index)
    number_of_each_categories[c] = number_of_rows
  max_value = sum(number_of_each_categories.values())/len(number_of_each_categories.values())
  for c in categories:
    if number_of_each_categories[c] >= max_value:
      continue
    tmp = df['Category'] == c
    indexes = df[tmp]
    n = max_value - number_of_each_categories[c]
    nn = int(n/len(indexes))
    df = df.append([indexes]*nn, ignore_index=True)
    nn = n - nn
    number_of_each_categories[c]+=n
    index = df.index[df["Category"] == c].tolist()
    number_of_rows = len(index)
  return df
      


df = information(df)

In [ ]:
x_train = df["Text"]
categories = np.array(list(set(df["Category"]))) 
y_train = df["Category"]
y_train_hot = pd.get_dummies(y_train)
print(len(y_train))
print(len(x_train))
print(y_train)

241863
241863
0              Science and Culture
1                            Sport
2                          Economy
3         Miscellaneous.World News
4                            Sport
                    ...           
241858                Social.Women
241859                Social.Women
241860                Social.Women
241861                Social.Women
241862                Social.Women
Name: Category, Length: 241863, dtype: object


In [ ]:
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def preprocessor(doc):
  norm = normalizer.normalize(doc)
  norm = re.sub("\d+", "", norm)
  norm = re.sub(r'\s*[A-Za-z]+\b', '' , norm)
  return norm

#Test
sample = x_train[0]
print(sample)
sample_pre = preprocessor(sample)
print(sample_pre)


خبرنامه دانشگاه علم و صنعت ايران 
شماره يازدهم از خبرنامه روابط عمومي دانشگاه علم و صنعت 
ايران در 48 صفحه با اخبار و مطالب علمي متنوعي از 
استادان اين دانشگاه منتشر شد. 
در اين شماره از خبرنامه، علاوه بر اختصاص صفحاتي چند به 
ديدار رئيس جمهوري از دانشگاه علم و صنعت و سخنراني در 
جمع دانشجويان اين دانشگاه، دانشكده مهندسي شيمي، 
پژوهشكده مكانيك خودرو و چند بخش ديگر اين دانشگاه معرفي 
شده است. 
تلفن روابط عمومي دانشگاه علم و صنعت ايران 7451180 و 
نشاني اينترنتي آن به قرار زير است: 
ir.ac.iust.WWW


خبرنامه دانشگاه علم و صنعت ایران 
شماره یازدهم از خبرنامه روابط عمومی دانشگاه علم و صنعت 
ایران در  صفحه با اخبار و مطالب علمی متنوعی از 
استادان این دانشگاه منتشر شد. 
در این شماره از خبرنامه، علاوه بر اختصاص صفحاتی چند به 
دیدار رئیس جمهوری از دانشگاه علم و صنعت و سخنرانی در 
جمع دانشجویان این دانشگاه، دانشکده مهندسی شیمی، 
پژوهشکده مکانیک خودرو و چند بخش دیگر این دانشگاه معرفی 
شده است. 
تلفن روابط عمومی دانشگاه علم و صنعت ایران  و 
نشانی اینترنتی آن به قرار زیر است:...



# Tokenize the documents and remove stopwords
sw = stopwords and special characters


In [ ]:
sw = stopwords()
def tokenizer(doc):
  words = word_tokenize(doc)
  result = [] 
  for w in words:
    if w in sw :
      continue
    result.append(w)
  return result

#Test
sample_token = tokenizer(sample_pre)
print(sample_token)

['خبرنامه', 'دانشگاه', 'علم', 'صنعت', 'ایران', 'شماره', 'یازدهم', 'خبرنامه', 'روابط', 'عمومی', 'دانشگاه', 'علم', 'صنعت', 'ایران', 'صفحه', 'اخبار', 'مطالب', 'علمی', 'متنوعی', 'استادان', 'دانشگاه', 'شماره', 'خبرنامه', 'اختصاص', 'صفحاتی', 'دیدار', 'رئیس', 'جمهوری', 'دانشگاه', 'علم', 'صنعت', 'سخنرانی', 'دانشجویان', 'دانشگاه', 'دانشکده', 'مهندسی', 'شیمی', 'پژوهشکده', 'مکانیک', 'خودرو', 'دانشگاه', 'معرفی', 'تلفن', 'روابط', 'عمومی', 'دانشگاه', 'علم', 'صنعت', 'ایران', 'نشانی', 'اینترنتی']


# Extracting the features of each document

In [ ]:
def get_features(input):
  features = [];
  for doc in tqdm(input):
    # doc = input[i]
    doc = preprocessor(doc)
    doc = tokenizer(doc)
    features.append(doc)
  return features

#Test
tmp = get_features(x_train[:6])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
x = get_features(x_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
x[0]

['خبرنامه',
 'دانشگاه',
 'علم',
 'صنعت',
 'ایران',
 'شماره',
 'یازدهم',
 'خبرنامه',
 'روابط',
 'عمومی',
 'دانشگاه',
 'علم',
 'صنعت',
 'ایران',
 'صفحه',
 'اخبار',
 'مطالب',
 'علمی',
 'متنوعی',
 'استادان',
 'دانشگاه',
 'شماره',
 'خبرنامه',
 'اختصاص',
 'صفحاتی',
 'دیدار',
 'رئیس',
 'جمهوری',
 'دانشگاه',
 'علم',
 'صنعت',
 'سخنرانی',
 'دانشجویان',
 'دانشگاه',
 'دانشکده',
 'مهندسی',
 'شیمی',
 'پژوهشکده',
 'مکانیک',
 'خودرو',
 'دانشگاه',
 'معرفی',
 'تلفن',
 'روابط',
 'عمومی',
 'دانشگاه',
 'علم',
 'صنعت',
 'ایران',
 'نشانی',
 'اینترنتی']

# Calculating tfidf

In [ ]:
def get_data(doc):
  # doc = preprocessor(doc)
  # doc = tokenizer(doc)
  return doc

tfidf_vectorize = TfidfVectorizer(
    analyzer='word',
    tokenizer=get_data,
    preprocessor=get_data,
    token_pattern=None,
    min_df = 20
    ) 

In [ ]:
train_tfidf = tfidf_vectorize.fit_transform(x)

# Preparing the tests

In [ ]:
test_df = read_data(test_url)
x_test = test_df["Text"]

In [ ]:
test = get_features(x_test)
test_tfidf = tfidf_vectorize.transform(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(test[0])

['هفت', 'اقلیم', 'آلودگی', 'هوا', 'پکن', 'تهدید', 'باافزایش', 'آلودگی', 'هوا', 'پکن', 'مقام\u200cهای', 'محیط', 'زیست', 'دراین', 'شهر', 'تاثیرات', 'آلودگی', 'هوا', 'برانسان', 'هشداردادند', 'گزارش', 'واحدمرکزی', 'خبربنقل', 'شبکه', 'تلویزیونی', 'چین', 'آلودگی', 'هوا', 'درپکن', 'درپی', 'خودروها', 'وتعداد', 'آسمان', 'خراش\u200cها', 'درپایتخت', 'چین', 'بوقوع', 'پیوسته_است', 'بنابه', 'مقام\u200cهای', 'محیط', 'زیست', 'درحال', 'حاضر', 'میزان', 'دی', 'اکسید', 'کربن', 'دراین', 'شهر', 'سطح', 'معمولی', 'فراتررفته', 'بطوری', 'سطح', 'شهر', 'تیره', 'وتار', 'تنفس', 'مردم', 'خفه', 'درحال', 'حاضر', 'نرخ', 'سالانه', 'خودرو', 'درپکن', 'درصد', 'ذکرشده', 'تاسال', 'درشهر', 'پکن', 'میلیون', 'وچهارصد', 'خودرو', 'حرکت', 'واین', 'درحالی', 'هرروزهزار', 'خودرو', 'شهرهای', 'پکن', 'تعدادی', 'بعلت', 'فرسوده', 'هوای', 'شهرپکن', 'آلوده', 'بنابه', 'کارشناسان', 'درصد', 'آلودگی', 'هوای', 'شهر', 'پکن', 'ازخودروها', 'معضل', 'شهر', 'پکن', 'سوزاندن', 'زغال', 'سنگ', 'زیرابسیاری', 'شهروندان', 'وکارخانجات', 'واقع', 'درحومه', 'پکن

# Selecting a model to classify 

# Logistic Regression

In [ ]:
#models
logistic_regression = Pipeline([
                               ('clf', linear_model.LogisticRegression(n_jobs=1, C=1e5))
]);
# cosine_similarity = (train_tfidf * train_tfidf.T).A

model = logistic_regression.fit(train_tfidf, y_train)       

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
results = model.predict(test_tfidf) 
save_result(results, "result.csv")

# LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
model2 = LinearSVC().fit(train_tfidf, y_train) 

In [ ]:
results2 = model2.predict(test_tfidf) 
save_result(results2, "result2.csv")

# MultinomialNB

In [ ]:
model1 =  MultinomialNB().fit(train_tfidf, y_train);
results3 = model2.predict(test_tfidf) 
save_result(results3, "result3.csv")